In [1]:
import pandas as pd
# For some reason we have to define this store before importing the citylearnwrapper...
store = pd.HDFStore("planning_model_data.h5")
import sys
 
# setting path
sys.path.append('../')
from citylearn_wrapper import CityLearnEnvWrapper
import csv

import numpy as np
import h5py

/home/doseok/miniconda3/envs/ActiveRL/lib/python3.10/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(


In [2]:
env_config = {
    "schema": "../data/citylearn_challenge_2022_phase_1/schema.json",
}
num_episodes = 1000

In [3]:
env = CityLearnEnvWrapper(env_config)

In [4]:
def collect_dataset(env, num_episodes, save_interval = 50):
    obss = []
    next_obss = []
    actions = []
    rews = []
    dones = []
    for i in range(num_episodes):
        print(f"Episode {i}, {len(obss)} steps collected")
        obs = env.reset()
        done = False
        while not done:
            action = env.action_space.sample()
            next_obs, rew, done, info = env.step(action)
            #data.append([obs, next_obs, action.tolist(), rew, done])
            obss.append(obs)
            next_obss.append(next_obs)
            actions.append(action)
            rews.append([rew])
            dones.append([done])
            obs = next_obs
        if i % save_interval == save_interval - 1:
            store_data([obss, next_obss, actions, rews, dones])
            obss = []
            next_obss = []
            actions = []
            rews = []
            dones = []
    return obss, next_obss, actions, rews, dones

def convert_to_df(data, prefix):
    return pd.DataFrame(data, columns = [f"{prefix}_{i}" for i in range(len(data[0]))])

def store_data(data):
    obss, next_obss, actions, rews, dones = data
    obss_df = convert_to_df(obss, "obs")
    next_obss_df = convert_to_df(next_obss, "next_obs")
    actions_df = convert_to_df(actions, "action")
    rews_df = convert_to_df(rews, "rew")
    dones_df = convert_to_df(dones, "done")
    #df = pd.concat([obss_df, next_obss_df, actions_df, rews_df, dones_df])
    store.append("obs", obss_df, format='t')
    store.append("next_obs", next_obss_df, format='t')
    store.append("actions", actions_df, format='t')
    store.append("rews", rews_df, format='t')
    store.append("dones", dones_df, format='t')

    #pd.to_pickle(df, filename)
    # with open(filename, "w") as f:
    #     writer = csv.writer(f, delimiter='|')
    #     writer.writerow(["obs", "next_obs", "action", "rew", "done"])
    #     writer.writerows(data)

In [5]:
data = collect_dataset(env, num_episodes)
#store_data(data, "planning_model_data.pkl")

Episode 0, 0 steps collected
Episode 1, 8759 steps collected
Episode 2, 17518 steps collected
Episode 3, 26277 steps collected
Episode 4, 35036 steps collected
Episode 5, 43795 steps collected
Episode 6, 52554 steps collected
Episode 7, 61313 steps collected
Episode 8, 70072 steps collected
Episode 9, 78831 steps collected
Episode 10, 87590 steps collected
Episode 11, 96349 steps collected
Episode 12, 105108 steps collected
Episode 13, 113867 steps collected
Episode 14, 122626 steps collected
Episode 15, 131385 steps collected
Episode 16, 140144 steps collected
Episode 17, 148903 steps collected
Episode 18, 157662 steps collected
Episode 19, 166421 steps collected
Episode 20, 175180 steps collected
Episode 21, 183939 steps collected
Episode 22, 192698 steps collected
Episode 23, 201457 steps collected
Episode 24, 210216 steps collected
Episode 25, 218975 steps collected
Episode 26, 227734 steps collected
Episode 27, 236493 steps collected
Episode 28, 245252 steps collected
Episode 29, 

In [6]:
store.close()